# image to .bin
- https://github.com/deepinsight/insightface/blob/master/src/data/lfw2pack.py

In [136]:
import cv2
import pickle
import numpy as np
import sys
from scipy import misc
import os
import tensorflow as tf
from scipy.io import loadmat
from easydict import EasyDict as edict
import os
import json
import numpy as np

# sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'align'))
#sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'common'))

# aligned된 사진으로 수행하기

## lfw.py

In [124]:
def add_extension(path):
    if os.path.exists(path+'.jpg'):
        return path+'.jpg'
    elif os.path.exists(path+'.png'):
        return path+'.png'
    else:
        raise RuntimeError('No file "%s" with extension png or jpg.' % path)
        
def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs)

def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    print('len(pairs)', len(pairs))
    for i,pair in enumerate(pairs):
        print(i, pair)
        if len(pair) == 3:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[2])))
            issame = True
        elif len(pair) == 4:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[2], pair[2] + '_' + '%04d' % int(pair[3])))
            issame = False
        if os.path.exists(path0) and os.path.exists(path1):    # Only add the pair if both paths exist
            path_list += (path0,path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs>0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)
    return path_list, issame_list

## lfw2pack.py

In [133]:
args = edict()
args.data_dir = '../data/lfw/lfw/'
args.image_size = '112,96'
args.output = '../data/lfw/lfw.bin'
args.model_path = './mtcnn_weights'
print(os.path.isdir(args.data_dir))

True


In [134]:
lfw_dir = args.data_dir
image_size = [int(x) for x in args.image_size.split(',')]
lfw_pairs = read_pairs(os.path.join(lfw_dir, 'pairs.txt'))
print(lfw_pairs.shape)

(6000,)


In [135]:
lfw_paths, issame_list = get_paths(lfw_dir, lfw_pairs)
lfw_bins = []

len(pairs) 6000
0 ['Abel_Pacheco', '1', '4']
1 ['Akhmed_Zakayev', '1', '3']
2 ['Akhmed_Zakayev', '2', '3']
3 ['Amber_Tamblyn', '1', '2']
4 ['Anders_Fogh_Rasmussen', '1', '3']
5 ['Anders_Fogh_Rasmussen', '1', '4']
6 ['Angela_Bassett', '1', '5']
7 ['Angela_Bassett', '2', '5']
8 ['Angela_Bassett', '3', '4']
9 ['Ann_Veneman', '3', '5']
10 ['Ann_Veneman', '6', '10']
11 ['Ann_Veneman', '10', '11']
12 ['Anthony_Fauci', '1', '2']
13 ['Antony_Leung', '1', '2']
14 ['Antony_Leung', '2', '3']
15 ['Anwar_Ibrahim', '1', '2']
16 ['Augusto_Pinochet', '1', '2']
17 ['Barbara_Brezigar', '1', '2']
18 ['Benjamin_Netanyahu', '1', '4']
19 ['Benjamin_Netanyahu', '4', '5']
20 ['Bernard_Law', '2', '3']
21 ['Bernard_Law', '3', '4']
22 ['Bertrand_Bonello', '1', '2']
23 ['Bill_Frist', '2', '9']
24 ['Bill_Frist', '4', '5']
25 ['Bob_Graham', '2', '4']
26 ['Bob_Graham', '3', '6']
27 ['Bob_Graham', '4', '6']
28 ['Bob_Graham', '5', '6']
29 ['Boris_Becker', '2', '6']
30 ['Brad_Johnson', '1', '3']
31 ['Brad_Johnson', '2'

305 ['Abel_Pacheco', '2', 'Jean-Francois_Lemounier', '1']
306 ['Afton_Smith', '1', 'Dwayne_Wade', '1']
307 ['Ahmad_Jbarah', '1', 'James_Comey', '1']
308 ['Akhmed_Zakayev', '2', 'Donna_Morrissey', '1']
309 ['Alan_Dershowitz', '1', 'Bertrand_Bonello', '1']
310 ['Alanis_Morissette', '1', 'Martin_Cauchon', '1']
311 ['Alexander_Lukashenko', '1', 'Heather_Chinnock', '1']
312 ['Alfonso_Cuaron', '1', 'Jason_Priestley', '1']
313 ['Alfonso_Cuaron', '1', 'Patty_Schnyder', '2']
314 ['Alfonso_Soriano', '1', 'Bill_Nelson', '2']
315 ['Alfonso_Soriano', '1', 'Julio_De_Brun', '1']
316 ['Alfonso_Soriano', '1', 'Patty_Schnyder', '3']
317 ['Alonzo_Mourning', '1', 'Cecilia_Cheung', '1']
318 ['Amber_Tamblyn', '2', 'Benjamin_Netanyahu', '1']
319 ['Amporn_Falise', '1', 'Joe_Pantoliano', '1']
320 ['Anders_Fogh_Rasmussen', '2', 'Johnson_Panjaitan', '2']
321 ['Andre_Bucher', '1', 'Joseph_Ralston', '1']
322 ['Andre_Bucher', '1', 'Maria_Garcia', '1']
323 ['Andrew_Gilligan', '1', 'Henry_Castellanos', '1']
324 ['And

513 ['Jennifer_Garner', '6', 'Mike_Duke', '1']
514 ['Jennifer_Renee_Short', '1', 'Taylor_Twellman', '1']
515 ['Jerry_Seinfeld', '1', 'Tim_Blake_Nelson', '1']
516 ['Jerry_Tarkanian', '1', 'Thomas_Rupprath', '1']
517 ['Jessica_Lange', '2', 'Sedigh_Barmak', '1']
518 ['Jim_Freudenberg', '1', 'Nigel_Redden', '1']
519 ['Jim_Freudenberg', '1', 'Tina_Pisnik', '1']
520 ['Jim_Haslett', '1', 'Tsutomu_Takebe', '1']
521 ['Jim_Otto', '1', 'Rafiq_Hariri', '1']
522 ['Jimmy_Gurule', '1', 'Terry_McAuliffe', '1']
523 ['Jodie_Foster', '3', 'Joe_Pantoliano', '1']
524 ['John_Herrington', '1', 'Luis_Ernesto_Derbez_Bautista', '2']
525 ['John_Richardson', '1', 'Yasushi_Akashi', '1']
526 ['John_Snow', '17', 'Se_Hyuk_Joo', '1']
527 ['Jonathan_Arden', '1', 'Joseph_Ralston', '1']
528 ['Jorge_Castaneda', '1', 'Robert_Fico', '1']
529 ['Jose_Rosado', '1', 'Micky_Arison', '1']
530 ['Joseph_Blatter', '1', 'Ronald_Kadish', '1']
531 ['Joseph_Ralston', '2', 'Juan_Pablo_Montoya', '5']
532 ['Joseph_Ralston', '2', 'Yoshiyuki

721 ['Howard_Dean', '1', '7']
722 ['Howard_Dean', '2', '8']
723 ['Howard_Dean', '4', '5']
724 ['Howard_Dean', '5', '12']
725 ['Hu_Jintao', '4', '14']
726 ['Hu_Jintao', '5', '12']
727 ['Hu_Jintao', '5', '15']
728 ['Hu_Jintao', '11', '13']
729 ['Ian_McKellen', '1', '3']
730 ['Ibrahim_Jaafari', '1', '2']
731 ['Isabella_Rossellini', '1', '3']
732 ['Ishaq_Shahryar', '1', '2']
733 ['Ismail_Merchant', '1', '2']
734 ['Jacques_Chirac', '2', '43']
735 ['Jacques_Chirac', '19', '28']
736 ['James_Cameron', '1', '3']
737 ['James_McGreevey', '1', '3']
738 ['James_McGreevey', '1', '4']
739 ['Jason_Alexander', '1', '2']
740 ['Jason_Kidd', '2', '8']
741 ['Jason_Kidd', '3', '8']
742 ['Jason_Kidd', '6', '8']
743 ['Jelena_Dokic', '1', '2']
744 ['Jelena_Dokic', '2', '4']
745 ['Jelena_Dokic', '2', '8']
746 ['Jelena_Dokic', '3', '4']
747 ['Jelena_Dokic', '4', '5']
748 ['Jelena_Dokic', '4', '7']
749 ['Jelena_Dokic', '4', '8']
750 ['Jelena_Dokic', '5', '6']
751 ['Jelena_Dokic', '7', '8']
752 ['Jennifer_Connelly

1055 ['Gao_Qiang', '2', 'Kenneth_Brill', '1']
1056 ['Gary_Stevens', '1', 'Wanda_Ilene_Barzee', '1']
1057 ['Geoff_Hoon', '1', 'Osama_bin_Laden', '3']
1058 ['Geoff_Hoon', '3', 'Richard_Tubb', '1']
1059 ['Geoff_Hoon', '5', 'Thomas_Stewart', '1']
1060 ['George_Brumley', '1', 'Manfred_Stolpe', '1']
1061 ['George_Brumley', '1', 'Ryan_Leaf', '1']
1062 ['George_Brumley', '2', 'Sergei_Yushenkov', '1']
1063 ['George_Papandreou', '2', 'Tatiana_Panova', '1']
1064 ['George_Papandreou', '4', 'Larry_Lindsey', '2']
1065 ['George_Papandreou', '4', 'Paul_LeClerc', '1']
1066 ['Ghassan_Elashi', '1', 'Rick_Rickert', '1']
1067 ['Gloria_Allred', '2', 'Roman_Tam', '1']
1068 ['Greg_Kinsey', '1', 'Priscilla_Presley', '1']
1069 ['Greg_Ostertag', '2', 'Kevin_Costner', '6']
1070 ['Greg_Owen', '1', 'Mark_Sisk', '1']
1071 ['Greg_Owen', '2', 'Mike_Smith', '1']
1072 ['Gretchen_Mol', '1', 'Isabella_Rossellini', '2']
1073 ['Gunilla_Backman', '1', 'Kathleen_Abernathy', '1']
1074 ['Hanan_Ashrawi', '1', 'Mario_Kreutzberger

...

4931 ['Jan_Ullrich', '1', '4']
4932 ['Jan_Ullrich', '2', '6']
4933 ['Jan_Ullrich', '3', '6']
4934 ['Jane_Pauley', '1', '2']
4935 ['Jason_Jennings', '1', '2']
4936 ['Javier_Weber', '1', '2']
4937 ['Jennifer_Rodriguez', '1', '2']
4938 ['Jeong_Se-hyun', '2', '6']
4939 ['Jeong_Se-hyun', '3', '7']
4940 ['Jeong_Se-hyun', '6', '8']
4941 ['Jeong_Se-hyun', '7', '9']
4942 ['Jeong_Se-hyun', '8', '9']
4943 ['Jo_Dee_Messina', '1', '2']
4944 ['Joe_Lieberman', '8', '11']
4945 ['Joe_Lieberman', '9', '10']
4946 ['Joe_Mantello', '1', '2']
4947 ['John_Allen_Muhammad', '2', '9']
4948 ['John_Allen_Muhammad', '5', '7']
4949 ['John_Allen_Muhammad', '6', '10']
4950 ['John_Blaney', '1', '2']
4951 ['John_Brady', '1', '2']
4952 ['John_Howard', '5', '15']
4953 ['John_Howard', '12', '17']
4954 ['Johnny_Tapia', '1', '2']
4955 ['Johnny_Tapia', '2', '3']
4956 ['Jorge_Valdano', '1', '2']
4957 ['Joseph_Deiss', '1', '3']
4958 ['Junichiro_Koizumi', '1', '53']
4959 ['Junichiro_Koizumi', '26', '55']
4960 ['Junichiro_Koizum

5151 ['Barry_Williams', '1', 'Wang_Fei', '1']
5152 ['Benazir_Bhutto', '4', 'Jane_Pauley', '2']
5153 ['Benazir_Bhutto', '5', 'Karen_Clarkson', '1']
5154 ['Bijan_Darvish', '1', 'Franco_Dragone', '1']
5155 ['Bijan_Darvish', '2', 'Frank_Taylor', '1']
5156 ['Bilal_Erdogan', '1', 'Lubomir_Zaoralek', '1']
5157 ['Bill_Self', '1', 'Yang_Jianli', '1']
5158 ['Bill_Simon', '1', 'Edward_Said', '2']
5159 ['Billy_Gilman', '1', 'Howard_Smith', '2']
5160 ['Billy_Sollie', '1', 'Mark_Swartz', '1']
5161 ['Billy_Sollie', '2', 'Hussein_Malik', '1']
5162 ['Blas_Ople', '1', 'Evan_Marriott', '1']
5163 ['Blas_Ople', '1', 'Felipe_Fernandez', '1']
5164 ['Blas_Ople', '1', 'Peter_Costello', '1']
5165 ['Bob_Cantrell', '1', 'Gerard_Kleisterlee', '1']
5166 ['Bob_Cantrell', '1', 'Mary_Landrieu', '3']
5167 ['Bob_Goldman', '1', 'Leigh_Winchell', '1']
5168 ['Bob_Goldman', '1', 'Maggie_Smith', '1']
5169 ['Boris_Berezovsky', '2', 'Leander_Paes', '1']
5170 ['Brad_Gushue', '1', 'Johannes_Rau', '1']
5171 ['Brad_Miller', '1', '

5317 ['John_Cruz', '1', 'Steven_Briggs', '1']
5318 ['John_Hartson', '1', 'Oscar_Elias_Biscet', '2']
5319 ['Joseph_Deiss', '1', 'Tony_Fernandes', '1']
5320 ['Joseph_Lopez', '1', 'Rick_Pitino', '1']
5321 ['Julian_Fantino', '1', 'Quin_Snyder', '1']
5322 ['Julio_Cesar_Chavez', '1', 'Tom_Brennan', '1']
5323 ['Junichi_Inamoto', '1', 'Oleg_Romantsev', '1']
5324 ['Junichiro_Koizumi', '20', 'Tony_Fernandes', '1']
5325 ['Karen_Clarkson', '1', 'Kristen_Breitweiser', '3']
5326 ['Karol_Kucera', '1', 'Qian_Qichen', '1']
5327 ['Karol_Kucera', '1', 'Turner_Gill', '1']
5328 ['Katrin_Cartlidge', '1', 'Tony_LaRussa', '1']
5329 ['Keith_Osik', '1', 'Pupi_Avati', '1']
5330 ['Kenneth_Cooper', '1', 'Norman_Mineta', '1']
5331 ['Kent_McCord', '1', 'Natasha_Henstridge', '1']
5332 ['Kevin_James', '1', 'Roy_Halladay', '1']
5333 ['Kevin_James', '1', 'Suzanne_Haik_Terrell', '1']
5334 ['Kevin_James', '1', 'Yang_Jianli', '1']
5335 ['Kevin_Spacey', '2', 'Ray_Evernham', '1']
5336 ['Koji_Uehara', '1', 'Rob_Moore', '1']
5

5595 ['Magdalena_Maleeva', '2', '3']
5596 ['Marcelo_Ebrard', '1', '2']
5597 ['Marcelo_Ebrard', '1', '3']
5598 ['Marcelo_Rios', '1', '3']
5599 ['Marcelo_Rios', '1', '4']
5600 ['Marcelo_Rios', '2', '3']
5601 ['Marcelo_Rios', '3', '4']
5602 ['Marcelo_Rios', '3', '5']
5603 ['Maria_Shriver', '1', '7']
5604 ['Maria_Shriver', '1', '8']
5605 ['Maria_Shriver', '2', '6']
5606 ['Maria_Shriver', '2', '8']
5607 ['Maria_Shriver', '3', '8']
5608 ['Marilyn_Monroe', '1', '2']
5609 ['Mario_Cipollini', '1', '2']
5610 ['Mark_Hurlbert', '1', '5']
5611 ['Mark_Hurlbert', '2', '3']
5612 ['Mark_Hurlbert', '2', '4']
5613 ['Martin_Scorsese', '2', '5']
5614 ['Martin_Scorsese', '3', '4']
5615 ['Marwan_Barghouthi', '1', '2']
5616 ['Michael_Bloomberg', '1', '12']
5617 ['Michael_Bloomberg', '3', '5']
5618 ['Michael_Bloomberg', '7', '11']
5619 ['Michael_Bloomberg', '11', '13']
5620 ['Michael_Chang', '1', '3']
5621 ['Michael_Chang', '5', '6']
5622 ['Michael_Jackson', '1', '6']
5623 ['Michael_Jackson', '2', '3']
5624 ['

5821 ['David_Oh', '1', 'Desiree_McKenzie', '1']
5822 ['David_Oh', '1', 'Joe_Garner', '1']
5823 ['David_Spade', '2', 'Denise_Locke', '1']
5824 ['David_Suazo', '1', 'Jane_Krakowski', '1']
5825 ['David_Wells', '1', 'Mary_Hill', '1']
5826 ['David_Wells', '1', 'Yannos_Papantoniou', '1']
5827 ['Dawna_LoPiccolo', '1', 'Jean-Claude_Juncker', '2']
5828 ['Debbie_Reynolds', '1', 'Nick_Nolte', '1']
5829 ['Deniz_Baykal', '1', 'Kathy_Bates', '1']
5830 ['Deniz_Baykal', '1', 'Robin_Wagner', '1']
5831 ['Desiree_McKenzie', '1', 'John_Danforth', '1']
5832 ['Dick_Devine', '1', 'Michael_Jackson', '9']
5833 ['Dinora_Rosales', '1', 'Robbie_Fowler', '1']
5834 ['Dirk_Kempthorne', '1', 'Janusz_Kaminski', '1']
5835 ['Donald_Evans', '2', 'Fred_Funk', '2']
5836 ['Dorothy_Wilson', '1', 'Jim_Thome', '1']
5837 ['Drew_Gooden', '1', 'Kimi_Raikkonen', '3']
5838 ['Dwain_Kyles', '1', 'Gregory_Peck', '1']
5839 ['Ed_Smart', '3', 'Jason_Gardner', '1']
5840 ['Edgar_Savisaar', '1', 'Kirk_Doerger', '1']
5841 ['Edward_Burns', '1

In [132]:
i = 0
for path in lfw_paths:
  with open(path, 'rb') as fin:
    _bin = fin.read()
    lfw_bins.append(_bin)
    #img = mx.image.imdecode(_bin)
    #img = nd.transpose(img, axes=(2, 0, 1))
    #lfw_data[i][:] = img
    i+=1
    if i%1000==0:
      print('loading lfw', i)

with open(args.output, 'wb') as f:
  pickle.dump((lfw_bins, issame_list), f, protocol=pickle.HIGHEST_PROTOCOL)


loading lfw 1000
loading lfw 2000
loading lfw 3000
loading lfw 4000
loading lfw 5000
loading lfw 6000
loading lfw 7000
loading lfw 8000
loading lfw 9000
loading lfw 10000
loading lfw 11000
loading lfw 12000
